In [1]:
import os
import sys
import gc
import logging
import pandas as pd
import numpy as np
from collections import defaultdict
from bitarray import bitarray
import pickle5
from tqdm import tqdm

from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import TruncatedSVD
from scipy.stats import multinomial

from sklearn.metrics import roc_auc_score

sys.path.append('../src')
import const
from utils import (
    seed_everything,
    Timer,
    reduce_mem_usage,
    # load_from_pkl,
#     save_as_pkl
)

from riiid_fe import riiidFE

# from preprocess_func import (
#     prep_tags,
#     part2lr,
#     merge_questions_w_prep,
#     prep_base,
#     add_modified_target_based_on_user_answer,
#     make_repeat_table,
#     make_elapsed_time_table
# )

# from model import riiidFE

In [2]:
# sys.path.append('/home/myaun/catboost/catboost/python-package')
# import catboost
# print('CAT Version', catboost.__version__)

INPUT_DIR = const.INPUT_DATA_DIR
FOLD_DIR = '../folds'
# EXP_DIR = '../exp'
# EXP_CONFIG = '001'
# sys.path.append(f'{EXP_DIR}/{EXP_CONFIG}')
import config

RANDOM_STATE = 46
FOLD_NAME = config.FOLD_NAME

use_features = config.use_features
FE_MODEL_PARAMS = config.FE_MODEL_PARAMS
CAT_PARAMS = config.CAT_PARAMS

w2v_feature_path = config.w2v_feature_path
g_feature_path = config.g_feature_path
corr_g_feature_path = config.corr_g_feature_path
ge_dw_feature_path = config.ge_dw_feature_path
ge_s2v_feature_path = config.ge_s2v_feature_path

SAVE_EXTACTED_FEATURES = False

# EXP_NAME = f'{FOLD_NAME}__CAT'
# if not os.path.exists(f'../features/{EXP_NAME}/'):
#     os.mkdir(f'../features/{EXP_NAME}/')

In [3]:
def prep_tags(x):
    return [int(i) for i in x.split()]


def part2lr(x):
    if x in [1, 2, 3, 4]:
        return 0
    if x in [5, 6, 7]:
        return 1


def merge_questions_w_prep(df, question):
    df = pd.merge(df, question[['question_id', 'part', 'tags', 'lr']], left_on='content_id', right_on='question_id', how='left')
    df['part'] = df['part'].fillna(0.0).astype('int8')
    df['tags'] = df['tags'].fillna('0')
    return df


def prep_base(df):
    # df['prior_question_elapsed_time_cat'] = pd.cut(df['prior_question_elapsed_time'], [0, 10000, 30000, 50000, 100000, 300000], labels=False).fillna(5).astype('int8')
    df['prior_question_elapsed_time'] = df['prior_question_elapsed_time'].fillna(-1).astype(int)
    df['prior_question_had_explanation'] = df['prior_question_had_explanation'].fillna(False).astype('int8')
    return df


def add_modified_target_based_on_user_answer(X_tra_wo_lec, question):
    # add modified target values 'weighted_answered_correctly', 'weighted_score'
    agg = X_tra_wo_lec.groupby(['content_id', 'user_answer'])['user_answer'].agg(['count']).reset_index()
    ccnt = X_tra_wo_lec['content_id'].value_counts().reset_index()
    ccnt.columns = ['content_id', 'c_count']

    agg = pd.merge(agg, ccnt, on='content_id', how='left')
    agg['choice_rate'] = agg['count'] / agg['c_count']
    agg = pd.merge(agg, question[['question_id', 'correct_answer']], right_on='question_id', left_on='content_id', how='left')
    agg['answered_correctly'] = (agg['user_answer'] == agg['correct_answer']) * 1

    # 'weighted_answered_correctly', 難しい問題に強い重み
    agg['weighted_answered_correctly'] = (1.0 - agg['answered_correctly'] * agg['choice_rate']) * agg['answered_correctly']

    # 'weighted_score', 不正解度合いによって重み付け
    agg2 = pd.DataFrame()
    for cid, tmp in tqdm(agg.groupby('content_id')):
        x = tmp[tmp.answered_correctly == 1]['choice_rate'].values.tolist() + tmp[tmp.answered_correctly != 1].sort_values('choice_rate', ascending=False)['choice_rate'].values.tolist()
        weighted_score = [1.0]
        for i in range(len(tmp) - 1):
            weighted_score.append(weighted_score[-1] - x[i])
        tmp['weighted_score'] = weighted_score
        agg2 = pd.concat([agg2, tmp])
    return agg2


def make_elapsed_time_table(X_tra_wo_lec, FOLD_NAME):
    rows = []
    for uid, udf in tqdm(X_tra_wo_lec[['user_id', 'timestamp', 'content_id', 'part', 'prior_question_had_explanation', 'prior_question_elapsed_time']].groupby('user_id')):
        udf['question_elapsed_time'] = udf['prior_question_elapsed_time'].shift(-1)
        udf = udf[~udf.question_elapsed_time.isna()]
        rows.extend(udf.values)

    et_table = pd.DataFrame(rows, columns=['user_id', 'timestamp', 'content_id', 'part', 'prior_question_had_explanation', 'prior_question_elapsed_time', 'question_elapsed_time'])
    et_table['content_id'] = et_table['content_id'].astype(int)
    et_table['part'] = et_table['part'].astype(int)
    et_table['prior_question_had_explanation'] = et_table['prior_question_had_explanation'].astype(int)
    et_table['question_elapsed_time'] = et_table['question_elapsed_time'].astype(int)
#     et_table.to_feather(f'../save/et_table_{FOLD_NAME}.feather')
    return et_table


def make_repeat_table(X_tra_wo_lec, FOLD_NAME):
    repeat_idx = []
    question_u_dict = {}
    for r, uid, cid, ans in tqdm(X_tra_wo_lec[['row_id', 'user_id', 'content_id', 'answered_correctly']].values):
        if uid not in question_u_dict:
            question_u_dict[uid] = np.zeros(13523, dtype=np.uint8)
        if question_u_dict[uid][cid] > 0:
            repeat_idx.append(r)
        question_u_dict[uid][cid] += 1
    repeat = X_tra_wo_lec[X_tra_wo_lec.row_id.isin(repeat_idx)]
    repeat = repeat.reset_index(drop=True)
#     repeat.to_feather(f'../save/repeat_{FOLD_NAME}.feather')
    return repeat

In [4]:
t = Timer()
with t.timer(f'fix seed RANDOM_STATE:{RANDOM_STATE}'):
    seed_everything(RANDOM_STATE)
    
with t.timer(f'read data'):
    X_tra = pd.read_pickle(f'{FOLD_DIR}/cv1_train.pickle').iloc[:100_000]
    X_val = pd.read_pickle(f'{FOLD_DIR}/cv1_valid.pickle').iloc[:2_500]
    question = pd.read_csv(f'{INPUT_DIR}/questions.csv')
    lecture = pd.read_csv(f'{INPUT_DIR}/lectures.csv')
    
with t.timer(f'Data split'):
    X_tra_wo_lec = X_tra[X_tra['content_type_id'] == False]
    X_val_wo_lec = X_val[X_val['content_type_id'] == False]

[fix seed RANDOM_STATE:46] done in 0 s (Total: 0.00 sec)
[read data] done in 4 s (Total: 4.27 sec)
[Data split] done in 0 s (Total: 4.27 sec)


In [5]:
with t.timer(f'Preprocess'):
    # preprocess
    question['lr'] = question['part'].apply(part2lr)   # part1 ~ 4 → 0, part5 ~ 7 → 1に変換

    X_tra = merge_questions_w_prep(X_tra, question)   # merge & fillna
    X_val = merge_questions_w_prep(X_val, question)

    X_tra = prep_base(X_tra)   # fillna
    X_val = prep_base(X_val)

    X_tra['tags'] = X_tra['tags'].fillna('0').apply(prep_tags)  # 0埋め & tagを[int, int, ...]の形に変換
    X_val['tags'] = X_val['tags'].fillna('0').apply(prep_tags)

    X_tra_wo_lec = prep_base(X_tra_wo_lec)   # fillna
    X_val_wo_lec = prep_base(X_val_wo_lec)

    X_tra_wo_lec = pd.merge(X_tra_wo_lec, question[['question_id', 'lr', 'part', 'tags']], left_on='content_id', right_on='question_id', how='left')
    X_val_wo_lec = pd.merge(X_val_wo_lec, question[['question_id', 'lr', 'part', 'tags']], left_on='content_id', right_on='question_id', how='left')

    agg = add_modified_target_based_on_user_answer(X_tra_wo_lec, question)   # content_idの選択肢ごとの重みdataframeを作成
    X_tra_wo_lec = pd.merge(X_tra_wo_lec, agg[['content_id', 'user_answer', 'weighted_answered_correctly', 'weighted_score']], on=['content_id', 'user_answer'], how='left')

    if os.path.exists(f'../save/et_table_{FOLD_NAME}.feather'):
        logging.info(f'skip make_elapsed_time_table')
        et_table = pd.read_feather(f'../save/et_table_{FOLD_NAME}.feather')
    else:
        et_table = make_elapsed_time_table(X_tra_wo_lec, FOLD_NAME)   # 回答時間を含めたdataframeの作成

    if os.path.exists(f'../save/rp_table_{FOLD_NAME}.feather'):
        logging.info(f'skip make_repeat_table')
        rp_table = pd.read_feather(f'../save/rp_table_{FOLD_NAME}.feather')
    else:
        rp_table = make_repeat_table(X_tra_wo_lec, FOLD_NAME)   #同じquestionの回答が2回目以降の部分だけを切り取ったdataframe

100%|██████████| 99067/99067 [00:00<00:00, 178488.19it/s]

[Preprocess] done in 31 s (Total: 35.68 sec)


In [6]:
# メモリ削減
with t.timer(f'reduce Mem'):
    X_tra = reduce_mem_usage(X_tra)
    X_val = reduce_mem_usage(X_val)
    X_tra_wo_lec = reduce_mem_usage(X_tra_wo_lec)
    X_val_wo_lec = reduce_mem_usage(X_val_wo_lec)

[reduce Mem] done in 0 s (Total: 35.72 sec)


In [7]:
with t.timer(f'init FE Model'):
    riiidFE = riiidFE(FE_MODEL_PARAMS)
    riiidFE.set_use_features(use_features)
    riiidFE.set_train_mn(X_tra_wo_lec)
    riiidFE.set_repeat_mn(rp_table)
    riiidFE.set_cat_te_dict(X_tra_wo_lec, question)   # target_encordingなどようの辞書を準備

Feature Num: 130
0.5724711558844015 0.1862932 0.5648155
0.7691119691119691 0.30617327551013307 0.5639839827949269
[init FE Model] done in 0 s (Total: 35.76 sec)


In [8]:
with t.timer(f'FE - loop features'):
    user_feat_df = riiidFE.add_user_feats(X_tra, add_feat=True, update_dict=True, val=False)
    del X_tra
    gc.collect()

    X_tra_wo_lec = pd.concat([X_tra_wo_lec, user_feat_df], axis=1)

    del user_feat_df
    gc.collect()
    X_tra_wo_lec = reduce_mem_usage(X_tra_wo_lec)

    # add tra only feaures
#     smooth = 10
#     tar = 'answered_correctly'
#     col_list = [
#         ['prev_part_s1', 'part'],
#         ['prev_part_s2', 'prev_part_s1', 'part'],
#         ['prev_part_s3', 'prev_part_s2', 'prev_part_s1', 'part'],
#         # ['prev_question_id_s1', 'question_id'],
#         # ['prev_question_id_s2', 'prev_question_id_s1', 'question_id'],
#     ]
#     for col in col_list:
#         tra_te_dict = pd.read_feather(f'../save/features_{FOLD_NAME}/{col[-1]}_sequence_s{len(col)-1}_tra_te_sm{smooth}.feather')
#         X_tra_wo_lec = pd.merge(X_tra_wo_lec, tra_te_dict, on=['row_id'], how='left')

#     # val only flag
    user_feat_df = riiidFE.add_user_feats(X_val, add_feat=True, update_dict=True, val=True)
    del X_val
    gc.collect()

    X_val_wo_lec = pd.concat([X_val_wo_lec, user_feat_df], axis=1)

    del user_feat_df
    gc.collect()
    X_val_wo_lec = reduce_mem_usage(X_val_wo_lec)

  0%|          | 1/100000 [00:00<00:24, 4068.19it/s]


IndexError: bitarray index out of range

In [14]:
bitarray(10)

bitarray('0001000011')

In [ ]:
X_tra_wo_lec.columns

In [ ]:
X_tra_wo_lec['timestamp_lag_diff_rolling10_median_each_user'].head(20)

In [ ]:
np.arange(10)[-7:]

In [ ]:
X_val_wo_lec

In [ ]:
riiidFE.user_prev_session_short_ts_dict

In [ ]:
# if __name__ == "__main__":
    
#     # mlflowの準備
#     mlflow.set_experiment(EXP_NAME)
#     mlflow.start_run()
#     run_id = mlflow.active_run().info.run_id

#     # ディレクトリとlogの準備
#     if not os.path.exists(f'../save/{EXP_NAME}_{run_id}/'):
#         os.mkdir(f'../save/{EXP_NAME}_{run_id}/')
#         os.mkdir(f'../save/{EXP_NAME}_{run_id}/features')
#     logging.basicConfig(filename=f'../save/{EXP_NAME}_{run_id}/logger.log', level=logging.INFO)

#     # seed値を固定
#     t = Timer()
#     with t.timer(f'fix seed RANDOM_STATE:{RANDOM_STATE}'):
#         seed_everything(RANDOM_STATE)

#     # 生データの読み込み
#     with t.timer(f'read data'):
#         folds = pd.read_feather(f'{FOLD_DIR}/train_folds_{FOLD_NAME}_v2.feather')
#         train = pd.read_feather(f'{INPUT_DIR}/train_v2.feather')
#         question = pd.read_csv(f'{INPUT_DIR}/questions.csv')
#         lecture = pd.read_csv(f'{INPUT_DIR}/lectures.csv')
    
#     # 生データの読み込み & 講義の削除
#     with t.timer(f'Data split'):
#         X_tra = train[train.row_id.isin(folds[folds.val == 0].row_id)]
#         X_val = train[train.row_id.isin(folds[folds.val == 1].row_id)]
#         X_tra_wo_lec = X_tra[X_tra.content_type_id == 0]
#         X_val_wo_lec = X_val[X_val.content_type_id == 0]
#         logging.info(f'X_tra:{len(X_tra)}, X_val:{len(X_val)}, X_tra_wo_lec{len(X_tra_wo_lec),}, X_val_wo_lec:{len(X_val_wo_lec)}')

#     del train
#     del folds
#     gc.collect()
    
#     # 前処理
#     with t.timer(f'Preprocess'):

#         # preprocess
#         question['lr'] = question['part'].apply(part2lr)   # part1 ~ 4 → 0, part5 ~ 7 → 1に変換

#         X_tra = merge_questions_w_prep(X_tra, question)   # merge & fillna
#         X_val = merge_questions_w_prep(X_val, question)

#         X_tra = prep_base(X_tra)   # fillna
#         X_val = prep_base(X_val)

#         X_tra['tags'] = X_tra['tags'].fillna('0').apply(prep_tags)  # 0埋め & tagを[int, int, ...]の形に変換
#         X_val['tags'] = X_val['tags'].fillna('0').apply(prep_tags)

#         X_tra_wo_lec = prep_base(X_tra_wo_lec)   # fillna
#         X_val_wo_lec = prep_base(X_val_wo_lec)

#         X_tra_wo_lec = pd.merge(X_tra_wo_lec, question[['question_id', 'lr', 'part', 'tags']], left_on='content_id', right_on='question_id', how='left')
#         X_val_wo_lec = pd.merge(X_val_wo_lec, question[['question_id', 'lr', 'part', 'tags']], left_on='content_id', right_on='question_id', how='left')

#         agg = add_modified_target_based_on_user_answer(X_tra_wo_lec, question)   # content_idの選択肢ごとの重みdataframeを作成
#         X_tra_wo_lec = pd.merge(X_tra_wo_lec, agg[['content_id', 'user_answer', 'weighted_answered_correctly', 'weighted_score']], on=['content_id', 'user_answer'], how='left')

#         if os.path.exists(f'../save/et_table_{FOLD_NAME}.feather'):
#             logging.info(f'skip make_elapsed_time_table')
#             et_table = pd.read_feather(f'../save/et_table_{FOLD_NAME}.feather')
#         else:
#             et_table = make_elapsed_time_table(X_tra_wo_lec, FOLD_NAME)   # 回答時間を含めたdataframeの作成

#         if os.path.exists(f'../save/rp_table_{FOLD_NAME}.feather'):
#             logging.info(f'skip make_repeat_table')
#             rp_table = pd.read_feather(f'../save/rp_table_{FOLD_NAME}.feather')
#         else:
#             rp_table = make_repeat_table(X_tra_wo_lec, FOLD_NAME)   #同じquestionの回答が2回目以降の部分だけを切り取ったdataframe

#         w2v_features = pd.read_feather(w2v_feature_path)
#         g_features = pd.read_feather(g_feature_path)
#         corr_g_features = pd.read_feather(corr_g_feature_path)
#         ge_dw_features = pd.read_csv(ge_dw_feature_path)
#         ge_s2v_features = pd.read_csv(ge_s2v_feature_path)

#     del agg
#     gc.collect()
        
#     # メモリ削減
#     with t.timer(f'reduce Mem'):
#         X_tra = reduce_mem_usage(X_tra)
#         X_val = reduce_mem_usage(X_val)
#         X_tra_wo_lec = reduce_mem_usage(X_tra_wo_lec)
#         X_val_wo_lec = reduce_mem_usage(X_val_wo_lec)

#     with t.timer(f'init FE Model'):
#         riiidFE = riiidFE(FE_MODEL_PARAMS)
#         riiidFE.set_use_features(use_features)
#         riiidFE.set_train_mn(X_tra_wo_lec)
#         riiidFE.set_repeat_mn(rp_table)
#         riiidFE.set_cat_te_dict(X_tra_wo_lec, question)   # target_encordingなどようの辞書を準備

#     with t.timer(f'FE - content_id features'):
#         riiidFE.extract_content_id_feat(
#             X_tra_wo_lec, rp_table, et_table, question,
#             w2v_features, g_features, corr_g_features, ge_dw_features
#         )
#         X_tra_wo_lec = pd.merge(X_tra_wo_lec, riiidFE.content_id_df, on='content_id', how='left')
#         X_val_wo_lec = pd.merge(X_val_wo_lec, riiidFE.content_id_df, on='content_id', how='left')

    # with t.timer(f'FE - content_id with features'):
    #     riiidFE.extract_content_idxxxprior_question_had_explanation_feat(X_tra_wo_lec, rp_table, et_table)
    #     X_tra_wo_lec = pd.merge(X_tra_wo_lec, riiidFE.content_idxxxprior_question_had_explanation_df, on=['content_id', 'prior_question_had_explanation'], how='left')
    #     X_val_wo_lec = pd.merge(X_val_wo_lec, riiidFE.content_idxxxprior_question_had_explanation_df, on=['content_id', 'prior_question_had_explanation'], how='left')

#     del w2v_features
#     del g_features
#     del corr_g_features
#     del ge_dw_features
#     del rp_table
#     del et_table
#     gc.collect()

#     with t.timer(f'reduce Mem'):
#         X_tra_wo_lec = reduce_mem_usage(X_tra_wo_lec)
#         X_val_wo_lec = reduce_mem_usage(X_val_wo_lec)

#     with t.timer(f'FE - loop features'):

#         user_feat_df = riiidFE.add_user_feats(X_tra, add_feat=True, update_dict=True, val=False)
#         del X_tra
#         gc.collect()

#         X_tra_wo_lec = pd.concat([X_tra_wo_lec, user_feat_df], axis=1)

#         del user_feat_df
#         gc.collect()
#         X_tra_wo_lec = reduce_mem_usage(X_tra_wo_lec)

#         # add tra only feaures
#         smooth = 10
#         tar = 'answered_correctly'
#         col_list = [
#             ['prev_part_s1', 'part'],
#             ['prev_part_s2', 'prev_part_s1', 'part'],
#             ['prev_part_s3', 'prev_part_s2', 'prev_part_s1', 'part'],
#             # ['prev_question_id_s1', 'question_id'],
#             # ['prev_question_id_s2', 'prev_question_id_s1', 'question_id'],
#         ]
#         for col in col_list:
#             tra_te_dict = pd.read_feather(f'../save/features_{FOLD_NAME}/{col[-1]}_sequence_s{len(col)-1}_tra_te_sm{smooth}.feather')
#             X_tra_wo_lec = pd.merge(X_tra_wo_lec, tra_te_dict, on=['row_id'], how='left')

#         # val only flag
#         user_feat_df = riiidFE.add_user_feats(X_val, add_feat=True, update_dict=True, val=True)
#         del X_val
#         gc.collect()

#         X_val_wo_lec = pd.concat([X_val_wo_lec, user_feat_df], axis=1)

#         del user_feat_df
#         gc.collect()
#         X_val_wo_lec = reduce_mem_usage(X_val_wo_lec)

#         # test
#         # riiidFE.add_user_feats(X_val, add_feat=False, update_dict=True)
#         # riiidFE.add_user_feats(X_val, add_feat=True, update_dict=False)
#         # print(fdsa)

#     if SAVE_EXTACTED_FEATURES is True:
#         X_tra_wo_lec.to_feather(f'../save/{EXP_NAME}_{run_id}/X_tra_wo_lec.feather')
#         X_val_wo_lec.to_feather(f'../save/{EXP_NAME}_{run_id}/X_val_wo_lec.feather')

# #     with t.timer(f'Save Model'):
# #         os.mkdir(f'../save/{EXP_NAME}_{run_id}/model')
# #         save_as_pkl(riiidFE, f'../save/{EXP_NAME}_{run_id}/features/riiidFE.pkl')

#     with t.timer(f'Train Model'):

#         model = catboost.CatBoostClassifier(
#             **CAT_PARAMS
#         )
#         logging.info(f'Sample Num: {len(X_tra_wo_lec)}')
#         logging.info(f'Feature Num: {len(use_features)}')

#         model.fit(
#             X_tra_wo_lec[use_features], X_tra_wo_lec['answered_correctly'].values,
#             cat_features=[],
#             # cat_features=categorical_features_index,
#             eval_set=(X_val_wo_lec[use_features], X_val_wo_lec['answered_correctly'].values),
#             use_best_model=True,
#             verbose=100
#         )

#         preds_val = model.predict_proba(X_val_wo_lec[use_features])[:, 1]
#         score = roc_auc_score(X_val_wo_lec['answered_correctly'].values, preds_val)

#     with t.timer(f'Feature importance + a'):
#         imp = model.get_feature_importance()
#         imp = pd.DataFrame(imp, index=use_features, columns=['importance']).sort_values('importance', ascending=False)

#         imp = imp.reset_index()
#         imp.columns = ['feat', 'importance']
#         f_unique = X_tra_wo_lec[use_features].nunique().reset_index()
#         f_unique.columns = ['feat', 'nunique']

#         imp = pd.merge(imp, f_unique, on='feat')
#         imp['min'] = X_tra_wo_lec[imp.feat.tolist()].min().values
#         imp['max'] = X_tra_wo_lec[imp.feat.tolist()].max().values
#         imp['mean'] = X_tra_wo_lec[imp.feat.tolist()].mean().values

# #     with t.timer(f'Save Model'):
# #         save_as_pkl(model, f'../save/{EXP_NAME}_{run_id}/model/cat_model.pkl')
# #         imp.to_csv(f'../save/{EXP_NAME}_{run_id}/importance_{run_id}.csv')
# #         save_mlflow(run_id, score)